In [17]:
# CONNECT TO DATABASE
import pymongo
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

### Find main emotion of a speech by using a greek sentiment lexicon <br> (https://github.com/MKLab-ITI/greek-sentiment-lexicon)

In [18]:
import spacy
import pandas as pd
import operator
indexes = {}
df = pd.read_csv('greek_sentiment_lexicon.tsv',sep='\t')
df = df.fillna('N/A')
for index, row in df.iterrows():
    df.at[index, "Term"] = row["Term"].split(' ')[0]
    indexes[df.at[index, "Term"]] = index
    
def main_emotion(speech, id,  indexes):
    results = {}
    text = speech
    subj_scores = {
        'OBJ': 0,
        'SUBJ-': 0.5,
        'SUBJ+': 1,
    }

    emotion_scores = {
        'N/A': 0,
        '1.0': 0.2,
        '2.0': 0.4,
        '3.0': 0.6,
        '4.0': 0.8,
        '5.0': 1,
    }

    nlp = spacy.load("el_core_news_sm")
    doc = nlp(text)
    matched_tokens = 0
    scorelist = [0 for _ in range(7)]
    for token in doc:
        lemmatized_token = token.lemma_
        if (lemmatized_token in indexes):
            indx = indexes[lemmatized_token]
            pos_flag = False
            for col in ["POS1", "POS2", "POS3", "POS4"]:
                if (token.pos_ == df.at[indx,col]):
                    pos_flag = True
                    break
            if (pos_flag == True):
                match_col_index = [int(s) for s in col if s.isdigit()][0]
                indexlist = ['Subjectivity', 'Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']
                for i in range(len(scorelist)):
                    if(i==0):
                        if(df.at[indx,indexlist[i]+str(match_col_index)]!='N/A'):
                            scorelist[i] += subj_scores[df.at[indx,indexlist[i]+str(match_col_index)]]
                    else:
                        if(df.at[indx, indexlist[i]+str(match_col_index)]!='N/A' and str(df.at[indx, indexlist[i]+str(match_col_index)])!= '0.0'):
                            scorelist[i] += emotion_scores[str(df.at[indx, indexlist[i]+str(match_col_index)])]
                matched_tokens+=1

    try:
        emotions = {'anger': scorelist[1], 'disgust': scorelist[2], 'fear':scorelist[3], 'happiness':scorelist[4], 'sadness': scorelist[5], 'surprise': scorelist[6]}
        emotion = max(emotions.items(), key=operator.itemgetter(1))[0]
        if (emotions[emotion] == 0):
            results[id] = ['N/A']
        else:
            results[id] = [emotion, str(emotions[emotion]*100/matched_tokens)]
    except:
        print('No matched tokens')
    return results

### Sentiments of speeches per parliament party

In [19]:
# FIND SENTIMENTS GIVEN THE SPEECHES OF A POLITICAL PARTY
def get_sentiments(speeches):
    party_sentiments = []
    for speech in speeches:
        results = main_emotion(speech['speech'], speech['_id'], indexes)
        party_sentiments.update(results)
        #count sentiments of all speeches of a party in the specified government
    sent_counter = {'surprise':0, 'happiness':0, 'disgust':0, 'anger':0, 'fear':0, 'sadness':0}
    for key in party_sentiments.keys():
        if(party_sentiments[key][0]!='N/A'):
            sent_counter[party_sentiments[key][0]] +=1 
    return sent_counter

In [ ]:
# FIND SENTIMENTS FOR SPEECHES OF EACH POLITICAL PARTY
sentiments_per_party = {}
counter = 0
parties = list(database.distinct("political_party"))
for party in parties:
    sentiments_per_party[party] = {}
    party_sentiments = []
    speeches = database.find({"political_party":party}, {"_id":1, "speech":1})
    sent_counter = get_sentiments(speeches)
    sentiments_per_party[party] = sent_counter
    
import pickle
with open('sentiments_per_party.pickle', 'wb') as handle:
    pickle.dump(sentiments_per_party, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# PLOT EXAMPLE
import pickle
with open('sentiments_per_party_sample.pickle', 'rb') as handle:
    sentiments_per_member = pickle.load(handle)
print(sentiments_per_member)
import plotly.graph_objects as go
sentiments = list(sentiments_per_member['δημοκρατικο κοινωνικο κινημα'].keys())
y = list(sentiments_per_member['δημοκρατικο κοινωνικο κινημα'].values())
fig = go.Figure([go.Bar(x=sentiments, y=y)])
fig.show()

{'δημοκρατικο κοινωνικο κινημα': {'surprise': 1247, 'happiness': 842, 'anger': 511, 'disgust': 101, 'fear': 40, 'sadness': 16}, 'ελληνικη λυση - κυριακος βελοπουλος': {'surprise': 461, 'happiness': 360, 'anger': 292, 'disgust': 78, 'fear': 10, 'sadness': 7}, 'ενωση κεντρωων': {'surprise': 759, 'happiness': 487, 'anger': 541, 'disgust': 36, 'fear': 12, 'sadness': 11}}


### Sentiments of speeches per parliament party per government

In [ ]:
parties = list(database.distinct("political_party"))
governments_per_party = {}
for party in parties:
    pipeline = [{'$match' : {'political_party':party}},
                    {'$group':{'_id':'$government'}}]
    governments = list(database.aggregate(pipeline))
    governments_per_party[party] = governments

In [ ]:
keys = list(governments_per_party.keys())

In [ ]:
party_sentiments_per_government = {}
counter = 0
#Find sentiments per party per government
for i in range(16, len(keys)):
    print(parties[i])
    party_sentiments_per_government[parties[i]] = {}
    party_sentiments = {}
    #for each government period the party is in the parliament
    for government in governments_per_party[parties[i]]:
        print("     "+government['_id'])
        #get speeches of party for that government
        speeches = database.find({"political_party":parties[i], "government":government['_id']}, {"_id":1, "speech":1})
        for speech in speeches:
            results = main_emotion(speech['speech'], speech['_id'], indexes)
            party_sentiments.update(results)
            
        #count sentiments of all speeches of a party in the specified government
        sent_counter = {'surprise':0, 'happiness':0, 'anger':0, 'disgust':0, 'anger':0, 'fear':0, 'sadness':0}
        for key in party_sentiments.keys():
            if(party_sentiments[key][0]!='N/A'):
                sent_counter[party_sentiments[key][0]] +=1 
                
        party_sentiments_per_government[parties[i]][government['_id']] = sent_counter
        import pickle

        with open('sentiments_per_party_per_government.pickle', 'wb') as handle:
            pickle.dump(party_sentiments_per_government, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
with open("sentiments_per_party_per_government.pickle", "rb") as input_file:
    sentiments = pickle.load(input_file)

In [45]:
print(sentiments.keys())

dict_keys(['ανεξαρτητοι ελληνες - πανος καμμενος', 'ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια', 'αντιπολιτευση', 'βουλη', 'δημοκρατικη ανανεωση', 'δημοκρατικη αριστερα', 'λαικη ενοτητα', 'λαικος ορθοδοξος συναγερμος', 'λαικος συνδεσμος - χρυση αυγη', 'μετωπο ευρωπαικης ρεαλιστικης ανυπακοης (μερα25)', 'νεα δημοκρατια'])


### Plot result examples

In [46]:
import pickle
with open("sentiments_per_party_per_government.pickle", "rb") as input_file:
    sentiments = pickle.load(input_file)
print(sentiments['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'])

{"['τσιπρα π. αλεξιου(26/01/2015-27/08/2015)']": {'surprise': 149, 'happiness': 95, 'anger': 47, 'disgust': 10, 'fear': 2, 'sadness': 2}, "['τσιπρα π. αλεξιου(21/09/2015-08/07/2019)']": {'surprise': 874, 'happiness': 626, 'anger': 446, 'disgust': 45, 'fear': 16, 'sadness': 8}}


In [47]:
vals = sentiments['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'].values()
y = []
for val in vals:
    y.append(list(val.values()))
import plotly.graph_objects as go
gov = list(sentiments['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'].keys())
vals = sentiments['ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια'].values()
fig = go.Figure(data=[
    go.Bar(name='surprise', x=gov, y=[y[0][0],y[1][0]]),
    go.Bar(name='happines', x=gov, y=[y[0][1],y[1][1]]),
    go.Bar(name='anger', x=gov, y=[y[0][2],y[1][2]]),
    go.Bar(name='disgust', x=gov, y=[y[0][3],y[1][3]]),
    go.Bar(name='fear', x=gov, y=[y[0][4],y[1][4]]),
    go.Bar(name='sadness', x=gov, y=[y[0][5],y[1][5]]),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()